In [ ]:
import numpy as np 
import pandas as pd
import random
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Conv2D, Dense, Dropout, Flatten
from tensorflow.keras import layers
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import xgboost as xgb
import lightgbm as lgb
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import KFold 
from sklearn.model_selection import cross_val_score

# %matplotlib inline   #쥬피터노트북에서 이미지 표시가능하게 하는 쥬피터노트북 매직함수
import matplotlib.pyplot as plt 
import seaborn as sns    
from pathlib import Path
import tensorflow as tf


import warnings
warnings.filterwarnings('ignore')

# **Data preparing**

In [ ]:
IMG_SIZE = 224
CHANNELS = 3
BATCH_SIZE = 16
EPOCHS = 10
SEED = 2021

DATA_DIR = '../input/petfinder-pawpularity-score/'
TRAIN_DIR = DATA_DIR + 'train/'
TEST_DIR = DATA_DIR + 'test/'

In [ ]:
# Configure Strategy. Assume TPU...if not set default for GPU/CPU
tpu = None
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except ValueError:
    # Enable XLA
    tf.config.optimizer.set_jit(enabled = "autoclustering")
    strategy = tf.distribute.get_strategy()

In [ ]:
# Load Train Data
sample_df = pd.read_csv(f'{DATA_DIR}train.csv')
sample_df['Id'] = sample_df['Id'].apply(lambda x: f'{TRAIN_DIR}{x}.jpg')

# Label value to be used for feature model 'classification' training.
sample_df['target_value'] = sample_df['Pawpularity'] / 100.

In [ ]:
sample_df = shuffle(sample_df, random_state=SEED)
train_size = int(len(sample_df)*0.8)
train_df = sample_df[:train_size]
validation_df = sample_df[train_size:]
train_df.head()

In [ ]:
training_data = tf.data.Dataset.from_tensor_slices((train_df['Id'].values, train_df['target_value'].values))
validation_data = tf.data.Dataset.from_tensor_slices((validation_df['Id'].values, validation_df['target_value'].values))

In [ ]:
def load_image_and_label_from_path(image_path, label):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=CHANNELS)
    img = tf.image.resize_with_pad(img, IMG_SIZE, IMG_SIZE)
    img = tf.image.adjust_brightness(img, 0.5)
    img = tf.image.adjust_saturation(img, 3)
    return img, label

AUTOTUNE = tf.data.experimental.AUTOTUNE #메모리 동적 할당을 위한 AUTOTUNE
training_data = training_data.map(load_image_and_label_from_path, num_parallel_calls=AUTOTUNE) #train 데이터를 불러옴
validation_data = validation_data.map(load_image_and_label_from_path,num_parallel_calls=AUTOTUNE) #validation 데이터를 불러옴

In [ ]:
#train 및 validation 데이터를 훈련하기 좋게 batch로 자름
training_data_batches = training_data.shuffle(buffer_size=1000).batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)
validation_data_batches = validation_data.shuffle(buffer_size=1000).batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)

# **이미지 데이터 훈련**

In [ ]:
model = tf.keras.Sequential()
effnet = tf.keras.applications.EfficientNetB0(include_top=False, classes=None, 
                                              weights="../input/efficientnet-keras-noisystudent-weights-b0b7/noisystudent/noisy.student.notop-b0.h5",
                                              input_shape = (IMG_SIZE, IMG_SIZE, CHANNELS))
effnet.trainable = True
model.add(effnet)
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.BatchNormalization())  
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(156, activation='relu'))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(1000, activation='relu'))
model.add(tf.keras.layers.Dense(186, activation='relu'))
model.add(tf.keras.layers.Dense(300, activation='relu'))
model.add(tf.keras.layers.Dense(560, activation='relu'))
model.add(tf.keras.layers.Dense(186, activation='relu'))
model.add(tf.keras.layers.Dense(186, activation='relu'))
model.add(tf.keras.layers.Dense(356, activation='relu'))
model.add(tf.keras.layers.Dense(186, activation='relu'))
model.add(tf.keras.layers.Dense(186, activation='relu'))
model.add(tf.keras.layers.Dense(186, activation='relu'))
model.add(tf.keras.layers.Dense(254, activation='relu'))
model.add(tf.keras.layers.Dense(186, activation='relu'))
model.add(tf.keras.layers.Dense(186, activation='relu'))
model.add(tf.keras.layers.Dense(186, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.summary()


In [ ]:
model.compile(optimizer='adam', loss = tf.keras.losses.BinaryCrossentropy(), metrics=[tf.keras.metrics.RootMeanSquaredError('rmse')])

In [ ]:
checkpoint =  tf.keras.callbacks.ModelCheckpoint(f'feature_model.h5',
                                              verbose = 1, 
                                              monitor = 'val_loss', 
                                              mode = 'min', 
                                              save_weights_only = True,
                                              save_best_only = True)
model.fit(training_data_batches, epochs = EPOCHS,callbacks = [checkpoint], validation_data = validation_data_batches, verbose=1 )

# **tabular data + cnn pawpularity**

In [ ]:
def make_tabular_data(df, features, drop_features = ['index', 'Id', 'Action', 'Human',  'Pawpularity']):
    features = pd.Series(np.squeeze(features), name='features')
    df = pd.concat([df.reset_index(), features], axis=1)
    df = df.drop(drop_features, axis=1)
    return df

# 'Action', 'Collage', 'Group',

In [ ]:
tabular_train = make_tabular_data(train_df, model.predict(training_data_batches))
tabular_valid = make_tabular_data(validation_df, model.predict(validation_data_batches))

In [ ]:
y_train = tabular_train['target_value']
X_train = tabular_train.drop(['target_value'], axis=1)
y_valid = tabular_valid['target_value']
X_valid = tabular_valid.drop(['target_value'], axis=1)

In [ ]:
t = pd.DataFrame(data={"col": train_df.dtypes.index, "type": train_df.dtypes}).reset_index(drop=True)


# **EDA**

In [ ]:
plt.figure(figsize=(15,15))
sns.heatmap(data = train_df.corr(), annot=True, 
fmt = '.2f', linewidths=.5, cmap='Blues')

예측하려는 타겟 벨류와 column들이 그렇게 관련이 없다
이미지를 어떤방식으로 처리하는가가 가장좋음

# Modeling

In [ ]:
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.08, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)

In [ ]:
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.08, n_estimators=20000,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11
                             )

In [ ]:
model_CBR = CatBoostRegressor(iterations=1900,
                              learning_rate=0.08,
                              depth=4,
                              l2_leaf_reg=20,
                              bootstrap_type='Bernoulli',
                              subsample=0.6, eval_metric='RMSE',
                              metric_period=50, od_type='Iter',
                              od_wait=45, random_seed=17,
                              allow_writing_files=False, verbose=False
                                )

In [ ]:
model_xgb.fit(X_train, y_train)
model_lgb.fit(X_train, y_train)
model_CBR.fit(X_train, y_train)

In [ ]:
pred_xgb = model_xgb.predict(X_valid)
pred_lgb = model_lgb.predict(X_valid)
pred_CBR = model_CBR.predict(X_valid)

In [ ]:
X_valid.head()

# **Ensemble**

In [ ]:
pred_ensemble_val = (pred_xgb + pred_lgb + pred_CBR)/3

# Evaluate

In [ ]:
# rmse
print(np.sqrt(mean_squared_error(pred_ensemble_val, y_valid)))
         
# xgb hyper parameter tune -> 0.207384233279449
# ensemble -> 0.20971427318649655 -> 20.96696

In [ ]:
# Load Test Data
test_df = pd.read_csv(f'{DATA_DIR}test.csv')
test_df['Id'] = test_df['Id'].apply(lambda x: f'{TEST_DIR}{x}.jpg')
test_df['Pawpularity'] = 0

# Summary
print(f'test_df: {test_df.shape}')
test_df.head()

In [ ]:
def load_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=CHANNELS)
    img = tf.image.resize_with_pad(img, IMG_SIZE, IMG_SIZE)
    img = np.reshape(img, [-1,IMG_SIZE, IMG_SIZE, CHANNELS])
    return img

In [ ]:
def test_predict(filepath):
    local_image = load_image(filepath)
    prediction = model.predict(local_image)
    return float(prediction)*100

In [ ]:
def predictions_over_image(filepath):
    predictions = []
    for path in filepath:
        predictions.append(test_predict(path))
    return predictions

In [ ]:
test_feature = predictions_over_image(test_df['Id'])
test_feature = pd.Series(test_feature, name='features')
tabular_test = pd.concat([test_df.reset_index(), test_feature], axis=1)
tabular_test = tabular_test.drop(['index', 'Id','Action', 'Human', 'Pawpularity'], axis=1)

In [ ]:
pred_xgb = model_xgb.predict(tabular_test)
pred_lgb = model_lgb.predict(tabular_test)
pred_CBR = model_CBR.predict(tabular_test)

In [ ]:
pred_ensemble_real = (pred_xgb + pred_lgb + pred_CBR)/3

In [ ]:
submission = pd.DataFrame(columns={"Id", "Pawpularity"})
submission['Id'] = test_df['Id'].map(lambda i : i.split('/')[-1].split('.')[0])
submission['Pawpularity'] = pred_ensemble_real
submission['Pawpularity'] = submission['Pawpularity'].map(lambda i: i*100)
submission.head()

In [ ]:
submission.to_csv('submission.csv', index = False)